In [108]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 



import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import countDistinct, col

In [109]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext
spark, sc

(<pyspark.sql.session.SparkSession at 0x115fc3588>,
 <SparkContext master=local[*] appName=pyspark-shell>)

In [110]:
df=pd.read_csv('data/training.csv')

In [111]:
df.head()

,user,movie,rating,timestamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977


In [112]:
del df['timestamp']

In [113]:
df.head()

,user,movie,rating
0,6040,858,4
1,6040,593,5
2,6040,2384,4
3,6040,1961,4
4,6040,2019,5


In [115]:
sdf=spark.createDataFrame(df)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:49585)

In [40]:
sdf.show()

+----+-----+------+
|user|movie|rating|
+----+-----+------+
|6040|  858|     4|
|6040|  593|     5|
|6040| 2384|     4|
|6040| 1961|     4|
|6040| 2019|     5|
|6040| 1419|     3|
|6040|  573|     4|
|6040| 3111|     5|
|6040|  213|     5|
|6040| 3505|     4|
|6040| 1734|     2|
|6040|  912|     5|
|6040|  919|     5|
|6040| 2503|     5|
|6040|  527|     5|
|6040|  318|     4|
|6040| 1252|     5|
|6040|  649|     5|
|6040| 3289|     5|
|6040|  759|     5|
+----+-----+------+
only showing top 20 rows



In [105]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [43]:
train, validate = sdf.randomSplit([0.8, 0.2])

In [44]:
als= ALS(itemCol='movie',userCol='user',ratingCol='rating',nonnegative=True,coldStartStrategy="drop")

In [107]:
#tuning the model
param_grid=ParamGridBuilder().addGrid(als.rank,[12,13,14]).addGrid(als.maxIter,[20,21,22]).addGrid(als.regParam,[.13,.14,.15]).build()

#Define evaluator as RMSE
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

#Build CrossValidation
tvs=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:49585)

In [102]:
model=tvs.fit(train)

Py4JJavaError: An error occurred while calling o17642.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 13646.0 failed 1 times, most recent failure: Lost task 1.0 in stage 13646.0 (TID 83818, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream$PeekInputStream.read(ObjectInputStream.java:2718)
	at java.io.ObjectInputStream$PeekInputStream.readFully(ObjectInputStream.java:2734)
	at java.io.ObjectInputStream$BlockDataInputStream.readInt(ObjectInputStream.java:3235)
	at java.io.ObjectInputStream.readHandle(ObjectInputStream.java:1739)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1590)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2342)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2266)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2124)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1625)
	at java.io.Obje

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50238)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/nivzz/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/nivzz/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/nivzz/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/Users/nivzz/a

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (mo

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (mo

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (mo

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (mo

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (mo

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49585)
Traceback (most recent call last):
  File "/Users/nivzz/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another e

In [89]:
best_model=model.bestModel

In [90]:
predictions=best_model.transform(validate)

In [91]:
rmse=evaluator.evaluate(predictions)

In [92]:
rmse

0.8782084542769883

In [93]:
print("RMSE",rmse)
print("Best Model\n")
print("Best Rank:", best_model.rank)
print("maxIter:", best_model._java_obj.parent().getMaxIter())
print("RegParam", best_model._java_obj.parent().getRegParam())

RMSE 0.8782084542769883
Best Model

Best Rank: 14
maxIter: 22
RegParam 0.13


In [94]:
df_pred=predictions.toPandas()

In [95]:
df_pred.head()

,user,movie,rating,prediction
0,3184,148,4,2.872095
1,1242,148,3,2.769016
2,3539,148,3,2.608525
3,1069,148,2,2.823630
4,3032,463,4,4.071906


In [96]:
del df_pred['rating']

In [97]:
df_pred.head()

,user,movie,prediction
0,3184,148,2.872095
1,1242,148,2.769016
2,3539,148,2.608525
3,1069,148,2.823630
4,3032,463,4.071906


In [100]:
df_pred.to_csv('/Users/nivzz/Downloads/recommend_matrix_fact_2.csv')

In [85]:
#for us to use to test our Predictions against a Validation Set
#format should has to be exactly:
'''
df_validation = ['user','movie','predicted_rating']
df_actual = ['user','movie','actual_rating']
Example on how you can rename cols
df_preds.columns= ['user', 'movie' ,'predicted_rating'] 
df_actual.columns = ['user', 'movie' ,'actual_rating'] 
'''
def compute_score(df_validation, df_actual):
    """Look at 5% of most highly predicted movies for each user.
    Return the average actual rating of those movies.
    """
    df = pd.merge(df_validation, df_actual, on=['user','movie']).fillna(1.0)
    #df = pd.concat([predictions.fillna(1.0), actual.actualrating], axis=1)
    # for each user
    g = df.groupby('user')
    # detect the top_5 movies as predicted by your algorithm
    top_5 = g.prediction.transform(
        lambda x: x >= x.quantile(.95)
    )
    # return the mean of the actual score on those
    return df.rating[top_5==1].mean()

In [98]:
compute_score(df_pred,df)

4.430954587581094

In [7]:
from pyspark.sql import SparkSession

# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()
...

# Convert a Pandas DF to a Spark DF
sdf = spark.createDataFrame(training) 

# Convert a Spark DF to a Pandas DF
df = sdf.toPandas()

In [62]:
df.head()

,user,movie,rating
0,6040,858,4
1,6040,593,5
2,6040,2384,4
3,6040,1961,4
4,6040,2019,5
